In [1]:
#Import the necessary libraries
import pandas as pd
from docx import Document
import os
from docx2pdf import convert
import win32com.client

In [2]:
# Function to load Excel data into a DataFrame
def load_excel_data(file_path: str, sheet_name: str = 'Sheet1') -> pd.DataFrame:
    """
    Load data from an Excel file into a Pandas DataFrame.
    
    Args:
        file_path (str): Path to the Excel file.
        sheet_name (str): Name of the Excel sheet to read data from.

    Returns:
        pd.DataFrame: DataFrame containing the Excel data.
    """
    return pd.read_excel(file_path, sheet_name=sheet_name)

In [3]:
# Function to create a personalized letter from the template
def create_personalized_letter(template_path: str, data: dict, output_path: str) -> None:
    """
    Create a personalized letter by replacing placeholders in the Word template.

    Args:
        template_path (str): Path to the Word document template.
        data (dict): Dictionary containing placeholders and corresponding replacement values.
        output_path (str): Path to save the personalized letter.
    """
    # Check if template file exists
    if not os.path.exists(template_path):
        raise FileNotFoundError(f"Template file '{template_path}' not found.")

    # Load the Word document template
    doc = Document(template_path)

    # Replace placeholders with actual data
    for paragraph in doc.paragraphs:
        for placeholder, replacement in data.items():
            if placeholder in paragraph.text:
                paragraph.text = paragraph.text.replace(placeholder, str(replacement))

    # Replace placeholders in tables
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for placeholder, replacement in data.items():
                    if placeholder in cell.text:
                        cell.text = cell.text.replace(placeholder, str(replacement))

    # Save the personalized letter
    doc.save(output_path)

In [4]:
#Function to count the number of files with a specific extension in a directory
def count_files_in_directory(directory: str, file_extension: str) -> int:
    """
    Count the number of files with a specific extension in the given directory.

    Args:
        directory (str): Path to the directory where files are stored.
        file_extension (str): Extension of the files to count.

    Returns:
        int: Number of files with the specified extension.
    """
    # List all files in the directory
    files = os.listdir(directory)
    # Filter and count files with the specific extension
    count = sum(1 for file in files if file.endswith(file_extension))
    return count

In [5]:
# Function to generate serial letters using data from the Excel table and include file counting
def generate_serial_letters(template_path: str, excel_path: str, output_dir: str, sheet_name: str = 'Sheet1') -> None:
    """
    Generate serial letters using a Word template and data from an Excel table.

    Args:
        template_path (str): Path to the Word document template.
        excel_path (str): Path to the Excel file containing the data.
        output_dir (str): Directory to save the personalized letters.
        sheet_name (str): Name of the Excel sheet to read data from.
    """
    # Check if Excel file exists
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"Excel file '{excel_path}' not found.")

    # Load Excel data
    df = load_excel_data(excel_path, sheet_name)

    # Ensure output directory exists
    os.makedirs(docx_folder, exist_ok=True)

    # Generate a personalized letter for each row in the DataFrame
    for _, row in df.iterrows():
        data = {f"{{{col}}}": val for col, val in row.items()}  # Placeholder format {ColumnName}
        output_path = os.path.join(docx_folder, f"{row['Name']}_letter.docx")
        create_personalized_letter(template_path, data, output_path)
        print(f"Created: {output_path}")

    # Count DOCX files
    docx_count = count_files_in_directory(output_dir, '.docx')
    print(f"Total DOCX files created: {docx_count}")

In [6]:
# Function to convert all .docx files in a directory to PDFs and include file counting
def convert_all_word_files_to_pdfs(word_dir: str, pdf_dir: str) -> None:
    def close_word():
        try:
            word = win32com.client.Dispatch("Word.Application")
            word.Quit()
        except Exception as e:
            print("Failed to close Word:", e)

    # Create the output folder if it doesn't exist
    if not os.path.exists(pdf_output_folder):
        os.makedirs(pdf_output_folder)

    # Convert each DOCX file in the input folder to PDF and save in the output folder
    try:
        convert(docx_folder, pdf_output_folder )
    finally:
        close_word()

    # Count PDF files
    pdf_count = count_files_in_directory(pdf_dir, '.pdf')
    
    print(f"Total PDF files created: {pdf_count}")
    print("Conversion complete.")

In [7]:
# Example usage to generate serial letters and convert them to PDFs
template_path = 'template.docx'
excel_path = 'data.xlsx'
docx_folder = 'letters'
pdf_output_folder = 'pdfs'

# Check that files exist
if not os.path.exists(template_path):
    raise FileNotFoundError(f"Template file '{template_path}' not found.")

if not os.path.exists(excel_path):
    raise FileNotFoundError(f"Excel file '{excel_path}' not found.")

# Generate personalized letters
generate_serial_letters(template_path, excel_path, docx_folder)

# Convert the letters to PDFs
convert_all_word_files_to_pdfs(docx_folder, pdf_output_folder )

Created: letters\John_letter.docx
Created: letters\Jane_letter.docx
Total DOCX files created: 2


  0%|          | 0/2 [00:00<?, ?it/s]

Total PDF files created: 2
Conversion complete.
